In [1]:
import os
import pandas as pd

from glob import glob  
from natsort import natsorted, ns

import geopy
import folium
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

import rasterio  
from rasterio.plot import show
from pyproj import Transformer
#from rasterio.windows import Window
import rioxarray 

import imageio
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from osgeo import gdal
from mayavi import mlab

import webbrowser
from IPython.display import Image


QSocketNotifier: Can only be used with threads started with QThread


In [ ]:
#address = "Gramyelaan 24 2960 Brecht"
#Provide an address with the format 'street name with number', 'postcode' and city'
address = input("Please Provide House Addres Here: ")

In [ ]:
def get_coordinates(address: str) -> float:
        """Function to get address coordinate and
            plot it on a map"""

        geolocator = Nominatim(user_agent="3D_house_app")
        location = geolocator.geocode(address)
        location.latitude, location.longitude
        location_lat_long = [location.latitude, location.longitude]
        return location_lat_long
        
def plot_address(fn):
    location_map = folium.Map(location=fn, zoom_start=25)
    folium.Marker(location=fn, popup=fn).add_to(location_map)
    location_map
    return location_map
plot_address(get_coordinates(address))

In [ ]:
latitude,longitude = get_coordinates(address)
latitude,longitude

In [ ]:
def transform_address(x,y):
    # transform to Belgium 'EPSG:31370' coordinate
    transformer = Transformer.from_crs("EPSG:4326", crs_to = 'EPSG:31370' ,always_xy=True)
    lat, lon = transformer.transform(longitude, latitude)
    return lat,lon

In [ ]:
lat,lon = transform_address(latitude,longitude)

In [ ]:
address_path = './House_address/Belgium_houses_address.csv'

In [ ]:
def get_coordinate_df(path,address):
    
    index = []
    
    df = pd.read_csv(path)
    
    for i in range(df.shape[0]):

        if df['Address'][i] == address :
            index.append(i)
            break
        continue

    xx, yy = df['EPSG:31370_x'][index][index[0]] ,df['EPSG:31370_y'][index][index[0]] 

    lat, lon = df['EPSG:4326_lat'][index][index[0]] ,df['EPSG:4326_lon'][index][index[0]] 
    
    return (xx, yy) , (lat, lon)

In [ ]:
(xx, yy) , (lat, lon) = get_coordinate_df(address_path,address)

In [ ]:
def get_tif(path):
    #Function to get all tif files and sort them
    tif_file =[]
    files = glob(path,recursive = True) 
    for file in files: 
        tif_file.append(file)
    tif_file = natsorted(tif_file, alg=ns.IGNORECASE)
    return tif_file

In [ ]:
dsm_path = './Map_files/DSM/**/*.tif'
DSM_tif = get_tif(dsm_path)
DSM_tif[:3]

In [ ]:
dtm_path = './Map_files/DTM/**/*.tif'
DTM_tif = get_tif(dtm_path)
DTM_tif[:3]

In [ ]:
bounds = []
for i in DSM_tif:
    src = rasterio.open(i)
    bounds.append(src.bounds)
bounds[:3]

In [ ]:
found_tif_path = []
for i,bound in enumerate(bounds,1):
    if (xx >= bound[0] and xx <= bound[2]) & \
        (yy >= bound[1] and yy <= bound[3]):
        found_tif_path.append('./Map_files/DSM/DHMVIIDSMRAS1m_k0'+ str(i) +'/GeoTIFF/DHMVIIDSMRAS1m_k0'+ str(i) + '.tif')
        print('The house is located in this tif :', 'DHMVIIDSMRAS1m_k0' + str(i) + '.tif')
    else:
        None

In [ ]:
found_tif_path [0]

In [ ]:
rast_df = rioxarray.open_rasterio(found_tif_path[0],masked=True,chunks=True)

In [ ]:
n = 20
coor1,coor2 = [(xx-n),(yy+n)],[(xx+n),(yy+n)]
coor3,coor4 = [(xx+n),(yy-n)] ,[(xx-n),(yy-n)]
geometries = [ {'type': 'Polygon', 'coordinates': [[coor1,coor2, coor3,coor4,coor1 ]]}]

In [ ]:
clipped = rast_df.rio.clip(geometries)

In [ ]:
clipped.plot()

In [ ]:
path = clipped.rio.to_raster(address +"_farrrr_dsm.tif",tiled=True, dtype="int32")

In [ ]:
chm = imageio.imread(address +"_farrrr_dsm.tif")

In [ ]:
nx,ny = chm.shape
size=100
x = np.linspace(0, size*2, nx)
y = np.linspace(0, size*2, ny)
yv,xv = np.meshgrid(x, y)
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(111, projection='3d')
chm3d=ax.plot_surface(xv,yv,chm,cmap='plasma',linewidth=0)
ax.set_title('CHM(Canopy Height Model)')
ax.set_xlabel('Distance (m)')
ax.set_ylabel('Distance (m)')
ax.set_zlabel('Elevation (m)')
ax.view_init(azim=35)
fig.colorbar(chm3d, shrink=0.3, aspect=10)
fig.savefig(address +'_3D.png', dpi=200) 
plt.show()

In [ ]:
import gemgis as gg
import pyvista as pv

fb = r'Gramyelaan 24 2960 Brecht_farrrr_dsm.tif'
mesh = gg.visualization.read_raster(path=fb,nodata_val=9999.0,name='Elevation [m]')

In [ ]:
topo = mesh.warp_by_scalar(scalars="Elevation [m]", factor=5.5)

sargs = dict(height=0.2, vertical=True, position_x=0.005, position_y=0.005)
#sargs = dict(interactive=True)

p = pv.Plotter(notebook=True)
p.add_mesh(mesh=topo, cmap='plasma', scalar_bar_args=sargs, clim=[-0, 40])
#p.add_mesh(mesh=topo, cmap='gist_ncar', scalar_bar_args=sargs, clim=[-0, 40])
p.set_background('black')
p.show_grid(color='black')
p.show()
# Remove from plotters so output is not produced in docs
pv.plotting._ALL_PLOTTERS.clear()